In [14]:
import splitfolders

In [15]:
# splitfolders.ratio(r'/home/snucse/ivp model lab/Breast Tumor Dataset(4)/Inbreast Dataset', output="Data1", seed=1337, ratio=(0.7, 0.3))

# CNN

In [26]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = r'/home/snucse/ivp model lab/Breast Tumor Dataset(4)/Inbreast Dataset'
validation_dir = r'/home/snucse/ivp model lab/Data/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)


Found 108 images belonging to 2 classes.
Found 23 images belonging to 2 classes.


In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
history = model.fit_generator(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
)

/tmp/ipykernel_13575/3351744738.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
4/4 [==============================] - 12s 3s/step - loss: 0.6691 - accuracy: 0.5741 - val_loss: 0.6797 - val_accuracy: 0.6471
Epoch 2/25
4/4 [==============================] - 9s 2s/step - loss: 0.6833 - accuracy: 0.6574 - val_loss: 0.6680 - val_accuracy: 0.6471
Epoch 3/25
4/4 [==============================] - 9s 2s/step - loss: 0.6600 - accuracy: 0.6574 - val_loss: 0.6644 - val_accuracy: 0.6471
Epoch 4/25
4/4 [==============================] - 10s 3s/step - loss: 0.6642 - accuracy: 0.6574 - val_loss: 0.6658 - val_accuracy: 0.6471
Epoch 5/25
4/4 [==============================] - 10s 2s/step - loss: 0.6575 - accuracy: 0.6574 - val_loss: 0.6659 - val_accuracy: 0.6471
Epoch 6/25
4/4 [==============================] - 10s 2s/step - loss: 0.6581 - accuracy: 0.6574 - val_loss: 0.6623 - val_accuracy: 0.6471
Epoch 7/25
4/4 [==============================] - 10s 2s/step - loss: 0.6897 - accuracy: 0.6574 - val_loss: 0.6615 - val_accuracy: 0.6471
Epoch 8/25
4/4 [====================

In [49]:
val_acc = history.history['val_accuracy'][-1]
print("Validation Accuracy:", val_acc)

Validation Accuracy: 0.6470588445663452


CNN train accuracy: 65.74, Validation accuracy: 64.71 

# ML

Pixel intensity

In [61]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

def extract_features(image):
    resized_image = cv2.resize(image, (100, 100)).flatten()
    return resized_image

def load_data(directory):
    X = []
    y = []
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            img = cv2.imread(os.path.join(class_dir, filename))
            if img is not None:
                features = extract_features(img)
                X.append(features)
                y.append(class_name)
    return np.array(X), np.array(y)

directory1 = r'/home/snucse/ivp model lab/Breast Tumor Dataset(4)/Inbreast Dataset/'
directory2 = r"/home/snucse/ivp model lab/Breast Tumor Dataset(4)/Inbreast Dataset/Malignant"

X1, y1 = load_data(directory1)

In [48]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=3)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7272727272727273


HOG

In [50]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from skimage import exposure

In [ ]:
def extract_hog_features(image):
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, channel_axis = 2)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd

def load_data(directory):
    X = []
    y = []
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            img = cv2.imread(os.path.join(class_dir, filename))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                hog_features = extract_hog_features(img)
                X.append(hog_features)
                y.append(class_name)
    return np.array(X), np.array(y)

directory1 = r'/home/snucse/ivp model lab/Breast Tumor Dataset(4)/Inbreast Dataset/'

X1, y1 = load_data(directory1)

lack time ran for more than 12 mins

Colour histogram

In [ ]:
def extract_color_histogram(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    hist_hue = cv2.calcHist([hsv_image], [0], None, [256], [0, 256])
    hist_saturation = cv2.calcHist([hsv_image], [1], None, [256], [0, 256])
    hist_value = cv2.calcHist([hsv_image], [2], None, [256], [0, 256])
    
    hist_features = np.concatenate([hist_hue, hist_saturation, hist_value]).flatten()
    
    return hist_features

def load_data(directory):
    X = []
    y = []
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        for filename in os.listdir(class_dir):
            img = cv2.imread(os.path.join(class_dir, filename))
            if img is not None:
                hist_features = extract_color_histogram(img)
                X.append(hist_features)
                y.append(class_name)
    return np.array(X), np.array(y)

directory1 = r'/home/snucse/ivp model lab/Breast Tumor Dataset(4)/Inbreast Dataset/'

X2, y2 = load_data(directory1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6818181818181818
